<a href="https://colab.research.google.com/github/BlackDevilOC/BlackDevilOC/blob/main/longscope_text_to_video_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%cd /content
!pip install diffusers==0.19.0 transformers accelerate imageio[ffmpeg] -U einops omegaconf decord safetensors
!pip install -q https://download.pytorch.org/whl/cu121/xformers-0.0.22.post7-cp310-cp310-manylinux2014_x86_64.whl
!git clone -b dev https://github.com/camenduru/Text-To-Video-Finetuning

!apt -y install -qq aria2
model = "polyware-ai/longscope"
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/{model}/raw/main/model_index.json -d /content/model -o model_index.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/{model}/raw/main/vae/config.json -d /content/model/vae -o config.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/{model}/resolve/main/vae/diffusion_pytorch_model.bin -d /content/model/vae -o diffusion_pytorch_model.bin
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/{model}/raw/main/unet/config.json -d /content/model/unet -o config.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/{model}/resolve/main/unet/diffusion_pytorch_model.bin -d /content/model/unet -o diffusion_pytorch_model.bin
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/{model}/raw/main/text_encoder/config.json -d /content/model/text_encoder -o config.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/{model}/resolve/main/text_encoder/pytorch_model.bin -d /content/model/text_encoder -o pytorch_model.bin
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/{model}/raw/main/scheduler/scheduler_config.json -d /content/model/scheduler -o scheduler_config.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/{model}/raw/main/tokenizer/merges.txt -d /content/model/tokenizer -o merges.txt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/{model}/raw/main/tokenizer/special_tokens_map.json -d /content/model/tokenizer -o special_tokens_map.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/{model}/raw/main/tokenizer/tokenizer_config.json -d /content/model/tokenizer -o tokenizer_config.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/{model}/raw/main/tokenizer/vocab.json -d /content/model/tokenizer -o vocab.json

In [ ]:
# Change directory to your project folder
%cd /content/Text-To-Video-Finetuning

# Define parameters
prompt = "waterfall"  #@param {type:"string"}
negative = "text, watermark, copyright, blurry, low resolution, blur, low quality"  #@param {type:"string"}

# Wrap in quotes for shell compatibility
prompt = f'"{prompt}"'
negative = f'"{negative}"'

# Model parameters
num_steps = 25  #@param {type:"raw"}
guidance_scale = 23  #@param {type:"raw"}
fps = 24  #@param {type:"raw"}
num_frames = 96  #@param {type:"raw"}

# Compose the command properly with all arguments quoted
command = f'''
python inference.py \
  -m "/content/model" \
  -p {prompt} \
  -n {negative} \
  -W 384 \
  -H 216 \
  -o "/content/outputs" \
  -d cuda \
  -x \
  -s {num_steps} \
  -g
